In [26]:
import pandas            as pd
import pandasql          as psql
import numpy             as np
import os
import datetime
import time
import math
import IPython

import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split


from   IPython.core.display           import display, HTML
from   decimal                        import Decimal

pd.set_option('display.max_columns' , 500)
pd.set_option('display.max_rows'    , 5000)
pd.set_option('display.max_colwidth', None)

print('Setting is done:')


#------------------------------------
print('Make the notebook html as wide as possible')
display(HTML("<style>.container { width:100% !important; }</style>"))
print()

script_dir = os.getcwd()
print(script_dir)

Setting is done:
Make the notebook html as wide as possible



G:\Meine Ablage\price_anomaly_detection


In [27]:
from sklearn.cluster import KMeans
from numpy import sqrt, array, random, argsort
from sklearn.preprocessing import scale
from sklearn.datasets import load_boston
import matplotlib.pyplot as plt


from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from sklearn import preprocessing
import pandas as pd
import numpy as np

from scipy.stats import multivariate_normal

In [28]:
df = pd.read_csv(script_dir +'/TSLA.CSV/')

In [38]:
#define paramter for feature normalization
def feature_normalize(dataset):
    mu = np.mean(dataset, axis=0)
    sigma = np.std(dataset, axis=0)
    return (dataset - mu) / sigma

#define the parameter and estimate the Gaussian distribution
def estimateGaussian(dataset):
    mu = np.mean(dataset, axis=0)
    sigma = np.cov(dataset.T)
    return mu, sigma

#define the multivariate Gaussian distribution
def multivariateGaussian(dataset, mu, sigma):
    p = multivariate_normal(mean=mu, cov=sigma)
    return p.pdf(dataset)

In [30]:
def selectThresholdByCV(probs,gt):
    best_epsilon = 0
    best_f1 = 0
    f = 0
    stepsize = (max(probs) - min(probs)) / 1000;
    epsilons = np.arange(min(probs),max(probs),stepsize)
    for epsilon in np.nditer(epsilons):
        predictions = (probs < epsilon)
        f = f1_score(gt, predictions, average = "binary")
        if f > best_f1:
            best_f1 = f
            best_epsilon = epsilon
    return best_f1, best_epsilon

In [41]:
train_data = df[['Date', 'High']]

In [48]:
n_training_samples = df.shape[0]
n_dim = df.shape[1]
train_data = df[['High']]

mu, sigma = estimateGaussian(train_data)
p = multivariateGaussian(train_data,mu,sigma)

p_cv = multivariateGaussian(train_data,mu,sigma)
fscore, ep = selectThresholdByCV(p_cv,gt_data)
outliers = np.asarray(np.where(p < ep))

# plt.figure()
# plt.xlabel("Latency (ms)")
# plt.ylabel("Throughput (mb/s)")
# plt.plot(tr_data[:,0],tr_data[:,1],"bx") plt.plot(tr_data[outliers,0],tr_data[outliers,1],"ro")
# plt.show()

NameError: name 'gt_data' is not defined

In [23]:
n_dim

7

In [19]:
mu

Open         5.848506e+01
High         5.978442e+01
Low          5.713246e+01
Close        5.854538e+01
Adj Close    5.854538e+01
Volume       3.047029e+07
dtype: float64

In [49]:
data = df[['Open']]

def select_threshold(probs, test_data):
    best_epsilon = 0
    best_f1 = 0
    f = 0
    stepsize = (max(probs) - min(probs)) / 1000;
    epsilons = np.arange(min(probs), max(probs), stepsize)
    for epsilon in np.nditer(epsilons):
        predictions = (probs < epsilon)
        f = f1_score(test_data, predictions, average='binary')
        if f > best_f1:
            best_f1 = f
            best_epsilon = epsilon

    return best_f1, best_epsilon

mu, sigma = estimate_gaussian(data)
p = multivariate_gaussian(data,mu,sigma)

#selecting optimal value of epsilon using cross validation
p_cv = multivariate_gaussian(crossval_data,mu,sigma)
fscore, ep = select_threshold(p_cv,test_data)
print(fscore, ep)

#selecting outlier datapoints
outliers = np.asarray(np.where(p < ep))

NameError: name 'crossval_data' is not defined

In [ ]:
# https://towardsdatascience.com/time-series-of-price-anomaly-detection-13586cd5ff46


####FRAGE: brauchen wir für Gaussian dristribution nicht eine 0/1 Variable? Wenn ja, welche ist das in unserem Fall?

df_class0 = df.loc[df['srch_saturday_night_bool'] == 0, 'price_usd']
df_class1 = df.loc[df['srch_saturday_night_bool'] == 1, 'price_usd']

envelope =  EllipticEnvelope(contamination = outliers_fraction) 
X_train = df_class0.values.reshape(-1,1)
envelope.fit(X_train)
df_class0 = pd.DataFrame(df_class0)
df_class0['deviation'] = envelope.decision_function(X_train)
df_class0['anomaly'] = envelope.predict(X_train)

envelope =  EllipticEnvelope(contamination = outliers_fraction) 
X_train = df_class1.values.reshape(-1,1)
envelope.fit(X_train)
df_class1 = pd.DataFrame(df_class1)
df_class1['deviation'] = envelope.decision_function(X_train)
df_class1['anomaly'] = envelope.predict(X_train)

df_class = pd.concat([df_class0, df_class1])
df['anomaly5'] = df_class['anomaly']
fig, ax = plt.subplots(figsize=(10, 6))
a = df.loc[df['anomaly5'] == -1, ('date_time_int', 'price_usd')] #anomaly
ax.plot(df['date_time_int'], df['price_usd'], color='blue')
ax.scatter(a['date_time_int'],a['price_usd'], color='red')
plt.show();